In [1]:
#use tensorboard
import pandas as pd
#from transformers import pipeline, set_seed
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# import nltk
from nltk.tokenize import sent_tokenize
from huggingface_hub import Repository
# from tqdm import tqdm
# nltk.download("punkt")


c:\Users\Rosha\code\textsummarizer\summarizerv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
nepali = pd.read_csv("Datasets/combined_data.csv")

In [4]:
nepali.head()

,formatted_article,nepali_summary
0,"""आर्टिफिसियल इन्टेलिजेन्स (एआई) को क्षेत्रमा क...",नेपालको स्वामित्वमा रहेको आर्टिफिसियल इन्टेलिज...
1,"""काठमाण्डाै – नेपाल आयल निगमले आगामी माघ १ गते...","नेपाल आयल निगमले विभिन्न डिपोमा पेट्रोल, डिजेल..."
2,"""नेपाली पर्यटकका लागि सबैभन्दा आकर्षक बाह्य पर...","भारत, संयुक्त अरब इमिरेट्स (युएई) र थाइल्याण्ड..."
3,"""काठमाण्डाै – सुनचाँदीको मूल्य माथी उकालो लागे...",नेपाल सुनचाँदी व्यवसायी महासंघका अनुसार आज छाप...
4,"""नेपाल विद्युत् प्राधिकरणले डेडिकेटेड फिडर र ट...",नेपाल विद्युत् प्राधिकरणले बारा–पर्सा औद्योगिक...


In [5]:
nepali.shape

(5048, 2)

In [6]:
nepali['dialogue'] = nepali.formatted_article
nepali['summary'] = nepali.nepali_summary

nepali.drop(columns= ['formatted_article','nepali_summary'], inplace=True)

In [7]:
english = pd.read_csv("Datasets/samsum-train.csv")


In [8]:
english.drop(columns ='id', inplace=True)

In [9]:
english.shape

(14732, 2)

UNDERSAMPLING ENGLISH DATA


In [10]:
from sklearn.utils import resample
english_sampled = english.sample(n=5048, random_state= 42)

In [11]:
language_pd= pd.concat([nepali[:500], english_sampled[:500]], ignore_index=True)

In [12]:
language_pd=language_pd.sample(frac=1, random_state= 50).reset_index(drop=True)

In [13]:
language_pd.keys()

Index(['dialogue', 'summary'], dtype='object')

In [14]:
language = Dataset.from_pandas(language_pd)

In [15]:
language

Dataset({
    features: ['dialogue', 'summary'],
    num_rows: 1000
})

In [16]:
language =language.filter()
language

Filter: 100%|██████████| 1000/1000 [00:00<00:00, 58018.12 examples/s]


Dataset({
    features: ['dialogue', 'summary'],
    num_rows: 1000
})

Tokenizing


In [17]:
model_checkpoints = 'google/mT5-small'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\Rosha\code\textsummarizer\summarizerv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\Rosha\code\textsummarizer\summarizerv\Lib\site-packages\transformers\convert_slow_tokenizer.py:551: UserWarning: T

In [18]:
input = tokenizer("नेपाल आयल निगमले स्थानीय तहको निर्वाचनका कारण पेट्रोलियम पदार्थको मूल्य वृद्धि गरेको छ । ")
input

{'input_ids': [11566, 45251, 1649, 4332, 64418, 1048, 14161, 7163, 259, 60315, 1131, 15649, 48995, 57123, 259, 13576, 21799, 82039, 1649, 117327, 24230, 58208, 1131, 34196, 7741, 259, 79254, 3081, 2661, 3596, 2237, 259, 378, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
tokenizer.convert_ids_to_tokens(input.input_ids)

['▁नेपाल',
 '▁आय',
 'ल',
 '▁नि',
 'गम',
 'ले',
 '▁स्थान',
 'ीय',
 '▁',
 'तह',
 'को',
 '▁निर्',
 'वाच',
 'नका',
 '▁',
 'कारण',
 '▁पे',
 'ट्रो',
 'ल',
 'ियम',
 '▁पद',
 'ार्थ',
 'को',
 '▁मूल',
 '्य',
 '▁',
 'वृद्ध',
 'ि',
 '▁गर',
 'ेको',
 '▁छ',
 '▁',
 '।',
 '</s>']

In [21]:
max_input = 1024
max_target_length = 100

def preprocess_function(txt):
    model_inputs = tokenizer(txt['dialogue'], max_length=max_input,truncation=True)
    labels = tokenizer(txt['summary'], max_length=max_target_length, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [22]:
tokenized_language = language.map(preprocess_function, batched = True)
tokenized_language

Map: 100%|██████████| 1000/1000 [00:00<00:00, 2263.72 examples/s]


Dataset({
    features: ['dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

FineTuning with Trainer API


In [25]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)


In [34]:
from sklearn.model_selection import train_test_split
train_test = tokenized_language.train_test_split(test_size=0.2, seed=42)
X_train = train_test['train']
X_test = train_test['test']


In [27]:
from  transformers import Seq2SeqTrainingArguments

batch_size = 4
num_train_epochs = 5

logging_steps = 10 #len(X_train)//batch_size
model_name = model_checkpoints.split("/")[-1]

In [28]:
args = Seq2SeqTrainingArguments("bilingular_model", eval_strategy="epoch", learning_rate=5.6e-5, per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps, push_to_hub=True)

In [30]:
import numpy as np
import rouge_score
def compute_metric(eval_pred):
    predictions , labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}


In [31]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [28]:
#from typing import Optional
# class CSCSeq2SeqTrainer(Seq2SeqTrainer):
    # def _save(self, output_dir: Optional[str] = None, state_dict=None):
    #     if state_dict ==None:
    #         state_dict = self.model.state_dict()
        
    #     for key, tensor in state_dict:
    #         if isinstance(torch, torch.Tensor):
    #             state_dict[key] = tensor.contiguous()
        

    #     super._save(output_dir, state_dict)       


In [37]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model = model,
    args= args,
    train_dataset=X_train,
    eval_dataset=X_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metric)

In [ ]:
trainer.train()

In [ ]:
import torch

def save_model_contiguous(model, save_directory):
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()

    model.save_pretrained(save_directory)

save_model_contiguous(model, "bilingual_model")


In [ ]:
torch.save(model.state_dict(), 'bilingual_model/bilingual_model.pth')

In [ ]:
tokenizer.save_pretrained("bilingual_model")

In [3]:
repo_name = "MKCL/Bilingual-mode_Nepali_and_English"
repo = Repository(local_dir="modelli", clone_from=repo_name)

Cloning https://huggingface.co/MKCL/Bilingual-mode_Nepali_and_English into local empty directory.


In [4]:
repo.push_to_hub()

Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.
remote: Password authentication in git is no longer supported. You must use a user access token or an SSH key instead. See https://huggingface.co/blog/password-git-deprecation
fatal: Authentication failed for 'https://huggingface.co/MKCL/Bilingual-mode_Nepali_and_English/'



OSError: remote: Password authentication in git is no longer supported. You must use a user access token or an SSH key instead. See https://huggingface.co/blog/password-git-deprecation
fatal: Authentication failed for 'https://huggingface.co/MKCL/Bilingual-mode_Nepali_and_English/'
